In [1]:
from collections import namedtuple
import struct
import sys
import os
import pathlib
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
pd.set_option('display.max_rows', 250)
import matplotlib.pyplot as plt
import h5py
import scipy
import scipy.stats as sp
from scipy import stats
import warnings
# for filename in os.listdir(os.getcwd()):
import shutil
from pathlib import Path
from collections import defaultdict
from datetime import datetime, date
from operator import itemgetter
from inspect import cleandoc
from typing import TypeVar, Callable, Union, overload, NamedTuple, Iterable, Tuple, List, Dict, TextIO
from typing_extensions import Literal
from collections import defaultdict
import pingouin as pg

In [2]:
def grabTag(name):
    name = name.rsplit('.', 1)[0]
    name = name + '.bin'
    #print(name)
    chip = New_tags_filenames[name]
    tagName = chip.get('tag')
    return tagName

In [3]:
header_format = 'QII'
header_struct_size = struct.calcsize(header_format)
replay_magic = 0xffeeffeeffaaffaa

FRAME_SKIP = 2

ReplayHeader = namedtuple('ReplayHeader', [
    'magic',
    'version',
    'frame_size'
])

frame_format = 'Qffffffhhhh'
frame_struct_size = struct.calcsize(frame_format)

ReplayFrame = namedtuple('ReplayFrame', [
    'frame_no',
    'bat_y',
    'ball_x',
    'ball_y',
    'action',
    'm1_baseline',
    'm2_baseline',
    'm1_activity',
    'm2_activity',
    'hit_count',
    'miss_count'
])

def read_header(f):
    buf = f.read(header_struct_size)
    tup = struct.unpack(header_format, buf)
    return ReplayHeader._make(tup)

def read_frames(f):
    while True:
        buf = f.read(frame_struct_size)
        if not buf:
            break
        tup = struct.unpack(frame_format, buf)
        frame = ReplayFrame._make(tup)
        yield frame

In [4]:
data.to_pickle('Data/paddlemovement.pkl')

NameError: name 'data' is not defined

In [ ]:
data
gdf = data
group = gdf['tag'].str.contains('ctl', case = False)
gdf['group'] = group.astype(int)
gdf['group']= np.where((gdf['tag'].str.contains('GFP', case = False) | 
                        (gdf['tag'].str.contains('tdt', case = False))), 2, gdf['group'])
gdf['group']= np.where((gdf['tag'].str.contains('in-silico', case = False)), 4, gdf['group'])
gdf['group']= np.where((gdf['tag'].str.contains('ngn2', case = False)), 2, gdf['group'])
gdf['group']= np.where((gdf['session']== '1') & (((gdf['group']== 0)) | 
                                                   ((gdf['group']== 2))), 3, gdf['group'])
gdf['group']= np.where((gdf['session']== '3') & (((gdf['group']== 0)) | 
                                                   ((gdf['group']== 2))), 3, gdf['group'])

gdf['distance'] = gdf['distance'].astype(float)
gdf['group_name'] = 99
gdf['group_name'] = np.where((gdf['group']== 0), "MCC", gdf['group_name'])
gdf['group_name'] = np.where((gdf['group']== 1), "CTL", gdf['group_name'])
gdf['group_name'] = np.where((gdf['group']== 2), "HCC", gdf['group_name'])
gdf['group_name'] = np.where((gdf['group']== 3), "RST", gdf['group_name'])
gdf['group_name'] = np.where((gdf['group']== 4), "IS", gdf['group_name'])
gdf['group_name'] = np.where((gdf['group']== 5), "NGN2", gdf['group_name'])
gdf['pltgroup'] = 99
df1 = gdf.groupby(['group_name', 'tag', "chip_id", "date", 'session']).mean()
df1 = df1.reset_index()
df1['pltgroup'] = np.where((df1['group']== 0), "3", df1['pltgroup'])
df1['pltgroup'] = np.where((df1['group']== 1), "0", df1['pltgroup'])
df1['pltgroup'] = np.where((df1['group']== 2), "4", df1['pltgroup'])
df1['pltgroup'] = np.where((df1['group']== 3), "2", df1['pltgroup'])
df1['pltgroup'] = np.where((df1['group']== 4), "1", df1['pltgroup'])
df1 = df1.sort_values(by=['pltgroup'])
df = df1
df['Zdistance'] = (df.distance - df.distance.mean())/df.distance.std(ddof=0) 
df['Zdistance'] = df['Zdistance'].abs()
df = df[df.Zdistance <= 3]

In [ ]:
check = df.groupby(['group']).agg(['mean','sem'])
check

In [ ]:
check = gdf.groupby(['group']).agg(['mean','sem'])
check

In [ ]:
#box plot for long rallies

labels = df.group_name.unique()
x_pos = np.arange(len(labels))
x = df['group_name']
y = df['distance']
sns.set(style="darkgrid")
ax = sns.boxplot(data=df, x=x, y=y, palette="Set2", showfliers=False, showmeans = True, 
                 meanprops={"markerfacecolor":"black", 
                       "markeredgecolor":"black",
                      "markersize":"5"})
ax.set_xticks(x_pos)
ax.set_xticklabels(labels)
#ax.set_title('Pong Performance over Time With All Features')
ax.set_ylabel('Average Paddle Movement (pixels)')
ax.set_xlabel('Group')

y, h, col = 90000, 1333, 'k'
x00, x01 = 0, .2
x10, x11 = 1, 1.2
x20, x21 = 2, 2.2
x30, x31 = 3, 3.2
x40, x41 = 4, 4.2

plt.text(x00, y-2.5, "###", ha='center', va='bottom', color=col,fontsize=8)
plt.text(x10, y-2.5, "###", ha='center', va='bottom', color=col,fontsize=8)
plt.text(x20, y-2.5, "###", ha='center', va='bottom', color=col,fontsize=8)

plt.text(x00, y-6000, "%%%", ha='center', va='bottom', color=col,fontsize=8)
plt.text(x10, y-6000, "%%%", ha='center', va='bottom', color=col,fontsize=8)
plt.text(x20, y-6000, "%%%", ha='center', va='bottom', color=col,fontsize=8)


plt.text(x00, y-12000, "@@@", ha='center', va='bottom', color=col,fontsize=8)
plt.text(x20, y-12000, "@@@", ha='center', va='bottom', color=col,fontsize=8)
plt.text(x20, y-18000, "^^", ha='center', va='bottom', color=col,fontsize=8)





sns.set(rc={'figure.figsize':(3,3)})
plt.savefig('/home/jovyan/work/dishpill/Analysis/Plots/paddlemove.pdf', bbox_inches='tight') 

In [ ]:
pg.anova(data=df, dv=('distance'), between='group', detailed = True)

In [ ]:
pg.pairwise_gameshowell(data =df, dv='distance', between='group_name').round(5)

In [ ]:
Pdf = df
Sdf = pd.read_pickle('Data/in_vitro_cells_sentience_corr.pkl')
Sdf = Sdf.rename(columns={"session_num": "session"})

In [ ]:
Sdf['group_name'] = 99
Sdf['group_name'] = np.where((Sdf['group']== 0), "MCC", Sdf['group_name'])
Sdf['group_name'] = np.where((Sdf['group']== 1), "CTL", Sdf['group_name'])
Sdf['group_name'] = np.where((Sdf['group']== 2), "HCC", Sdf['group_name'])
Sdf['group_name'] = np.where((Sdf['group']== 3), "RST", Sdf['group_name'])
Sdf['group_name'] = np.where((Sdf['group']== 4), "IS", Sdf['group_name'])
Sdf['group_name'] = np.where((Sdf['group']== 5), "NGN2", Sdf['group_name'])
Sdf['pltgroup'] = 99
Sdf = Sdf.groupby(['group_name', 'tag', "chip_id", "date", 'session']).mean()
Sdf = Sdf.reset_index()
Sdf['pltgroup'] = np.where((Sdf['group']== 0), "3", Sdf['pltgroup'])
Sdf['pltgroup'] = np.where((Sdf['group']== 1), "0", Sdf['pltgroup'])
Sdf['pltgroup'] = np.where((Sdf['group']== 2), "4", Sdf['pltgroup'])
Sdf['pltgroup'] = np.where((Sdf['group']== 3), "2", Sdf['pltgroup'])
Sdf['pltgroup'] = np.where((Sdf['group']== 4), "1", Sdf['pltgroup'])
Sdf = Sdf.sort_values(by=['pltgroup'])

In [ ]:
Pdf['chip_id'] = Pdf['chip_id'].map(lambda x: ''.join([i for i in x if i.isdigit()]))
Pdf['date'] = pd.to_datetime(Pdf['date'])
Sdf['date'] = pd.to_datetime(Sdf['date'])
Pdf['session'] = pd.to_numeric(Pdf['session'])
Sdf['session'] = pd.to_numeric(Sdf['session'])
Pdf['session'] = Pdf['session'].astype('int64')
Sdf['session'] = Sdf['session'].astype('int64')
Pdf['chip_id'] = Pdf['chip_id'].astype('int64')
Sdf['chip_id'] = Sdf['chip_id'].astype('int64')


In [ ]:
alldf = Sdf.merge(Pdf, how='inner', on = ['group', 'chip_id', 'date', 'session'])


In [ ]:
alldf.group_name_x.unique()

In [ ]:
corr = alldf#[((alldf.group == 0) | (alldf.group == 2))]
pg.pairwise_corr(corr, method='pearson', columns=['hit_count']).round(3)

In [ ]:
x = alldf['distance']
y = alldf['hit_count']
f, ax = plt.subplots()
#ax.xaxis.update_units(x)
sns.regplot(x=x, y=y, line_kws={"color": "red"})
sns.set(style="darkgrid")
#ax.set_xlim([-1, 6])
ax.set_ylabel('Average Rally Length')
ax.set_xlabel('Average Distance Paddle Travels (pixels)')
ax.grid(False)
sns.set(rc={'figure.figsize':(3,3)})
plt.savefig('/home/jovyan/work/dishpill/Analysis/Plots/distancereg.pdf', bbox_inches='tight') 